# Testing the Resnet NN on CFAIR-10 Images:

## Import libraries, setup dataset:

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np
from sklearn.metrics import confusion_matrix
import torchvision
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [20]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

image_w = 32
image_h = 32

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) #RGB mean and std. dev

testing_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load testing data
testing_dataset_augment = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testing_generator = torch.utils.data.DataLoader(testing_dataset, batch_size=500, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


## Re-Declare ResNet Structure:

In [3]:
class Conv_block_batch_norm(nn.Module):
  def __init__(self, channels_in, channels_out):
      super().__init__()  
      ks = 3
      self.conv = nn.Conv2d(channels_in, channels_out,kernel_size=ks, padding=1, stride=1) #Passing = 1 -> no shrinking
      self.bn = nn.BatchNorm2d(channels_out)
  
  def forward(self, x):       # Forward pass which defines how the layers relate the input x to the output out
      x = self.conv(x)
      x = F.gelu(x)         # We could relu here, I'm not sure if it helps.
      x = self.bn(x)                               
      x = F.max_pool2d(x,kernel_size=2)
      x = F.gelu(x)
      return x

In [4]:
class Conv_residual_block_gelu(nn.Module):
    def __init__(self, channels_in_out):
        super().__init__()  
        ks = 3
        self.conv1 = nn.Conv2d(channels_in_out, channels_in_out, kernel_size=ks, padding=1, stride=1)
        self.conv2 = nn.Conv2d(channels_in_out, channels_in_out, kernel_size=ks, padding=1, stride=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.bn = nn.BatchNorm2d(channels_in_out)

    def forward(self, x):       # Forward pass which defines how the layers relate the input x to the output out
        x = self.bn(x)
        resid = x
        x = self.conv1(x)
        x = F.gelu(x)    
        
        x = self.conv2(x)
        x = F.gelu(x)     
        return x + resid


In [5]:
class Conv_Res_Net_basic(nn.Module):
  def __init__(self, img_w, img_h, num_classes):
    super().__init__()  
    num_kernels1 = 64
    num_kernels2 = 128
    num_kernels3 = 256
    #-- Block 1
    self.conv_block1 = Conv_block_batch_norm(3,num_kernels1)     
    img_w = img_w//2 #Max pool shrinks size 2x
    img_h = img_h//2
    self.residual_block_1a = Conv_residual_block_gelu(num_kernels1)  #Kernel size does not change
    self.residual_block_1b = Conv_residual_block_gelu(num_kernels1)
    self.residual_block_1c = Conv_residual_block_gelu(num_kernels1)
    self.residual_block_1c = Conv_residual_block_gelu(num_kernels1)

    #-- Block 2
    self.conv_block2 = Conv_block_batch_norm(num_kernels1,num_kernels2)   
    img_w = img_w//2 #Max pool shrinks size 2x
    img_h = img_h//2
    self.residual_block_2a = Conv_residual_block_gelu(num_kernels2)  #Kernel size does not change
    self.residual_block_2b = Conv_residual_block_gelu(num_kernels2)
    self.residual_block_2c = Conv_residual_block_gelu(num_kernels2)
    self.residual_block_2d = Conv_residual_block_gelu(num_kernels2)  #Kernel size does not change
    self.residual_block_2e = Conv_residual_block_gelu(num_kernels2)
    # self.residual_block_2f = Conv_residual_block_gelu(num_kernels2)
  
    #-- Block 3
    self.conv_block3 = Conv_block_batch_norm(num_kernels2,num_kernels3)  
    img_w = img_w//2 #Max pool shrinks size 2x
    img_h = img_h//2
    self.residual_block_3a = Conv_residual_block_gelu(num_kernels3)  #Kernel size does not change
    self.residual_block_3b = Conv_residual_block_gelu(num_kernels3)
    self.residual_block_3c = Conv_residual_block_gelu(num_kernels3)
    self.residual_block_3d = Conv_residual_block_gelu(num_kernels3)
    # self.residual_block_3e = Conv_residual_block_gelu(num_kernels3)
    # self.residual_block_3f = Conv_residual_block_gelu(num_kernels3)

    #-- Classifier
    self.flattened_dim = (img_w*img_h) * num_kernels3
    self.linear = nn.Linear(self.flattened_dim, num_classes)
  
  def forward(self, x):       # Forward pass which defines how the layers relate the input x to the output out
    #-- Block 1 --
    x = self.conv_block1(x)
    x = self.residual_block_1a(x)   # You can have as many of these fixed width blocks as you want in a row
    x = self.residual_block_1b(x)
    x = self.residual_block_1c(x)   
    #-- Block 2 --
    x = self.conv_block2(x)
    x = self.residual_block_2a(x)   # You can have as many of these fixed width blocks as you want in a row
    x = self.residual_block_2b(x)
    x = self.residual_block_2c(x)   
    x = self.residual_block_2d(x)   # You can have as many of these fixed width blocks as you want in a row
    x = self.residual_block_2e(x)
    # x = self.residual_block_2f(x)  
    #-- Block 3 --
    x = self.conv_block3(x)
    x = self.residual_block_3a(x)   # You can have as many of these fixed width blocks as you want in a row
    x = self.residual_block_3b(x)
    x = self.residual_block_3c(x)   
    x = self.residual_block_3d(x)  
    # x = self.residual_block_3e(x)   
    # x = self.residual_block_3f(x)   

    #-- Linear Classifier --
    x = x.view(-1, self.flattened_dim)
    x = self.linear(x)
    
    return x

In [6]:
use_cuda = torch.cuda.is_available()
print("Using GPU?:",use_cuda)
if (use_cuda):
  print("GPU Name:",torch.cuda.get_device_name())
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

Using GPU?: True
GPU Name: Tesla T4


In [28]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load the test dataset
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified


## Load and test the model:

In [31]:
network = Conv_Res_Net_basic(image_w, image_h, 10)
network.load_state_dict(torch.load('./best_model.pt'))
# network = network.to(device)

<All keys matched successfully>

In [36]:
network.eval()
allPreds, allTargs = [], []

# Predict
with torch.no_grad():
    for img, label in test_dataset:
        output = network(img.unsqueeze(0))
        _, pred = torch.max(output.data, 1)
        allPreds.append(pred.item())
        allTargs.append(label)

# Get accuracy, display confusion matrix
accuracy = sum([1 if allPreds[i] == allTargs[i] else 0 for i in range(len(allPreds))]) 
accuracy/= len(allPreds)
print(f"Accuracy: {accuracy}")

cm = confusion_matrix(allTargs, allPreds)
print(f"Confusion matrix:\n{cm}")

Accuracy: 0.7891
Confusion matrix:
[[779  11  66  22  30   1   6  12  57  16]
 [ 13 900   3   6   4   3   1   3  21  46]
 [ 48   3 752  48  59  34  27  22   5   2]
 [  8   7  72 642  77  88  68  21  10   7]
 [ 13   2  70  52 777  13  21  42   9   1]
 [  8   3  50 184  47 644  14  43   4   3]
 [  3   5  57  37  22   9 857   4   4   2]
 [  7   0  21  45  55  26   5 835   3   3]
 [ 43  14  11   7   6   0   4   6 901   8]
 [ 45  76  11  13   7   2   1  11  30 804]]


In [35]:
print(classes)

('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


### The model had an accuracy of 80% when tested in its own file and ~78-79 when loaded into a separate file.